In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error

data_path = 'UrbanEdgeApparel.csv'
data = pd.read_csv(data_path)

In [2]:
data['Order Date'] = pd.to_datetime(data['Order Date'])

monthly_data = data.groupby([
    pd.Grouper(key='Order Date', freq='M'), 
    'Product ID', 
    'Company ID', 
    'Shipping Country'
]).agg({
    'Order ID': 'count',
    'Total Selling Price': 'sum',
    'Product Quantity': 'sum'
}).reset_index()

monthly_data.rename(columns={
    'Order ID': 'Total Sales',
    'Total Selling Price': 'Total Sell Price',
    'Product Quantity': 'Total Quantity'
}, inplace=True)

/var/folders/83/176ss48d3c9f59p39cy9n9lc0000gn/T/ipykernel_65883/2567455480.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.Grouper(key='Order Date', freq='M'),


In [3]:
def create_lag_features(df, lags, columns):
    for column in columns:
        for lag in range(1, lags + 1):
            df[f'{column}_lag_{lag}'] = df.groupby(['Product ID', 'Company ID', 'Shipping Country'])[column].shift(lag)
    return df

def create_moving_average_features(df, windows, columns):
    for column in columns:
        for window in windows:
            df[f'{column}_ma_{window}'] = df.groupby(['Product ID', 'Company ID', 'Shipping Country'])[column].transform(lambda x: x.rolling(window, min_periods=1).mean())
    return df

In [4]:
lag_columns = ['Total Sales', 'Total Sell Price', 'Total Quantity']
ma_windows = [3, 6, 12]
monthly_data = create_lag_features(monthly_data, lags=3, columns=lag_columns)
monthly_data = create_moving_average_features(monthly_data, windows=ma_windows, columns=lag_columns)

monthly_data.dropna(inplace=True)

features = [col for col in monthly_data.columns if col not in ['Order Date', 'Product ID', 'Company ID', 'Shipping Country', 'Total Sales', 'Total Sell Price', 'Total Quantity']]
target_sales = 'Total Sales'
target_price = 'Total Sell Price'
target_quantity = 'Total Quantity'

In [5]:

X = monthly_data[features]
y_sales = monthly_data[target_sales]
y_price = monthly_data[target_price]
y_quantity = monthly_data[target_quantity]

X_train, X_test, y_sales_train, y_sales_test = train_test_split(X, y_sales, test_size=0.2, random_state=42)
_, _, y_price_train, y_price_test = train_test_split(X, y_price, test_size=0.2, random_state=42)
_, _, y_quantity_train, y_quantity_test = train_test_split(X, y_quantity, test_size=0.2, random_state=42)

rf_sales = RandomForestRegressor(n_estimators=100, random_state=42)
rf_price = RandomForestRegressor(n_estimators=100, random_state=42)
rf_quantity = RandomForestRegressor(n_estimators=100, random_state=42)

rf_sales.fit(X_train, y_sales_train)
rf_price.fit(X_train, y_price_train)
rf_quantity.fit(X_train, y_quantity_train)

y_sales_pred = rf_sales.predict(X_test)
y_price_pred = rf_price.predict(X_test)
y_quantity_pred = rf_quantity.predict(X_test)

mape_sales = mean_absolute_percentage_error(y_sales_test, y_sales_pred)
mape_price = mean_absolute_percentage_error(y_price_test, y_price_pred)
mape_quantity = mean_absolute_percentage_error(y_quantity_test, y_quantity_pred)

print(f'Total Sales MAPE: {mape_sales}')
print(f'Total Selling Price MAPE: {mape_price}')
print(f'Total Quantity MAPE: {mape_quantity}')

Total Sales MAPE: 0.01837458756432952
Total Selling Price MAPE: 0.13382459089819285
Total Quantity MAPE: 0.12106527109747402


In [6]:

future_dates = pd.date_range(start=monthly_data['Order Date'].max() + pd.DateOffset(months=1), periods=24, freq='M')

future_data_template = monthly_data[['Product ID', 'Company ID', 'Shipping Country']].drop_duplicates()
future_data_template = future_data_template.assign(key=1).merge(pd.DataFrame({'Order Date': future_dates, 'key': 1}), on='key').drop('key', axis=1)

for feature in features:
    future_data_template[feature] = np.nan

def generate_future_data(last_known_data, future_template, model_sales, model_price, model_quantity, features):
    future_data = future_template.copy()
    for i, row in future_data.iterrows():
        subset = last_known_data[(last_known_data['Product ID'] == row['Product ID']) &
                                 (last_known_data['Company ID'] == row['Company ID']) &
                                 (last_known_data['Shipping Country'] == row['Shipping Country'])]
        if not subset.empty:
            last_row = subset.iloc[-1]
            for feature in features:
                if 'lag' in feature:
                    lag_num = int(feature.split('_')[-1])
                    if lag_num == 1:
                        future_data.at[i, feature] = last_row[target_sales]
                    else:
                        future_data.at[i, feature] = last_row[feature.replace(f'lag_{lag_num}', f'lag_{lag_num-1}')]
                elif 'ma' in feature:
                    window = int(feature.split('_')[-1])
                    values = subset[target_sales].values[-window:]
                    future_data.at[i, feature] = values.mean() if len(values) > 0 else 0
    X_future = future_data[features].fillna(0)
    future_data['Total Sales'] = model_sales.predict(X_future)
    future_data['Total Sell Price'] = model_price.predict(X_future)
    future_data['Total Quantity'] = model_quantity.predict(X_future)
    return future_data

future_predictions = generate_future_data(monthly_data, future_data_template, rf_sales, rf_price, rf_quantity, features)


total_company_predictions = future_predictions.groupby('Order Date').agg({
    'Total Sales': 'sum',
    'Total Sell Price': 'sum',
    'Total Quantity': 'sum'
}).reset_index()

product_predictions = future_predictions.groupby(['Order Date', 'Product ID']).agg({
    'Total Sales': 'sum',
    'Total Sell Price': 'sum',
    'Total Quantity': 'sum'
}).reset_index()

company_predictions = future_predictions.groupby(['Order Date', 'Company ID']).agg({
    'Total Sales': 'sum',
    'Total Sell Price': 'sum',
    'Total Quantity': 'sum'
}).reset_index()

country_predictions = future_predictions.groupby(['Order Date', 'Shipping Country']).agg({
    'Total Sales': 'sum',
    'Total Sell Price': 'sum',
    'Total Quantity': 'sum'
}).reset_index()

print("Total Company Predictions:\n", total_company_predictions.head())
print("\nProduct Predictions:\n", product_predictions.head())
print("\nCompany Predictions:\n", company_predictions.head())
print("\nCountry Predictions:\n", country_predictions.head())

/var/folders/83/176ss48d3c9f59p39cy9n9lc0000gn/T/ipykernel_65883/1426993183.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start=monthly_data['Order Date'].max() + pd.DateOffset(months=1), periods=24, freq='M')


Total Company Predictions:
   Order Date  Total Sales  Total Sell Price  Total Quantity
0 2024-06-30      5054.65         7138.0838          3697.7
1 2024-07-31      5054.65         7138.0838          3697.7
2 2024-08-31      5054.65         7138.0838          3697.7
3 2024-09-30      5054.65         7138.0838          3697.7
4 2024-10-31      5054.65         7138.0838          3697.7

Product Predictions:
   Order Date  Product ID  Total Sales  Total Sell Price  Total Quantity
0 2024-06-30   Prod_1000       294.87          260.1115          148.21
1 2024-06-30  Prod_10021         5.00            9.9765            4.00
2 2024-06-30   Prod_1003        18.00           39.8350           16.00
3 2024-06-30   Prod_1005        70.00          151.1540           61.20
4 2024-06-30   Prod_1007        30.00           74.9370           30.00

Company Predictions:
   Order Date     Company ID  Total Sales  Total Sell Price  Total Quantity
0 2024-06-30   Company_1021         1.00            2.5075 